# Setting up required packages and code from old repo

In [ ]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach

In [ ]:
!pip -q install keras
!pip -q install colorlog
!pip -q install keras-radam
!pip -q install lmdb
!pip -q install Pillow

In [ ]:
import os
os.chdir('./Generative_Models/') # adress to git dir
print(os.getcwd())
from training.generators.file_image_generator import create_image_lists, get_generators
import numpy as np
import lmdb
import pickle
import sys
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from utils.data_and_files.file_utils import get_file_path

# Store images in lmdb

In [ ]:
images_dir = './data/.pokemon/' #Folder to images
validation_percentage = 30
valid_format = 'png'

In [ ]:
image_lists = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose = 1
)

In [ ]:
#Wrapper class for dataset
class Pokemon_Image:
    def __init__(self, image, label):
        self.channels = image.shape[2]
        self.size = image.shape[:2]
        self.image = image.tobytes()
        self.label = label #additional data to be stored (make it string)

    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.float32) #pay attention if you  don't use create_image_lists
        return images.reshape(*self.size, self.channels)     #then dtype will be different

In [ ]:
def store_single_lmdb(filename, img , index, label, num_images):
    """ Stores a wrapper to LMDB.
    """
    map_size = num_images * img.nbytes * 10
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
          # All key-value pairs need to be Strings
          value = Pokemon_Image(img, label)
          key = f"{index:08}"
          txn.put(key.encode("ascii"), pickle.dumps(value))

    env.close()

In [ ]:
def store_lmdb(image_lists, image_dir, 
               lmdb_dir = '/content/LMDB',category='training', target_size=None,
               color_mode='rgb', save_prefix='', save_format='png'):
  classes = list(image_lists.keys())
  num_class = len(classes)
  class2id = dict(zip(classes, range(len(classes))))
  id2class = dict((v, k) for k, v in class2id.items())
 
  y = None
  X = None

  for label_name in classes:
    num_images = len(image_lists[label_name][category])
    print('Storing '+ str(num_images)+lmdb_dir+os.sep+'_{}'.format(category))
    for index, _ in enumerate(image_lists[label_name][category]):
        img_path = get_file_path(image_lists,
                                  label_name,
                                  index,
                                  image_dir,
                                  category)
        img = img_to_array(
                        load_img(
                        img_path,
                        grayscale=color_mode=='grayscale',
                        target_size=target_size
                        )
                    )
        name, _ = os.path.splitext(img_path)
        vid_img_arr = name.split(sep=os.sep)[-1:]
        y = [np.array(vid_img_arr)]     
        name =  lmdb_dir+os.sep+'_{}'.format(category)
        
        store_single_lmdb(index = index, filename=name, img = img,label = y ,num_images = num_images)

In [ ]:
!rm -rf ./LMDB/
!mkdir ./LMDB/
# save_to_dir=None,
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='training',target_size=None,color_mode='rgb',save_prefix='',save_format='png')
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='validation',target_size=None,color_mode='rgb',save_prefix='',save_format='png')

# Done on ubuntu system is local machine does not have tar 
 - Install git bash then u can use such commands.
 - Go to windows store and download Ubuntu emulator wich just installs bash
 - On colab these should work

 ## For Training

In [22]:
!tar -cvjf pokemon_training.tar.bz2 /home/azeghost/git/Generative_Models/LMDB/_training/* 

tar: Removing leading `/' from member names
/home/azeghost/git/Generative_Models/LMDB/_training/data.mdb
/home/azeghost/git/Generative_Models/LMDB/_training/lock.mdb


In [23]:
!split -b 10M pokemon_training.tar.bz2 "pokemon_training.tar.part"
#split -b <max size> <name of zip or dir to zip/name> <split file name beginning>

In [24]:
#Check if they are created
!ls -lh pokemon_training.tar.part*

-rw-r--r-- 1 azeghost azeghost  10M Jul  1 22:35 pokemon_training.tar.partaa
-rw-r--r-- 1 azeghost azeghost  10M Jul  1 22:35 pokemon_training.tar.partab
-rw-r--r-- 1 azeghost azeghost  10M Jul  1 22:35 pokemon_training.tar.partac
-rw-r--r-- 1 azeghost azeghost  10M Jul  1 22:35 pokemon_training.tar.partad
-rw-r--r-- 1 azeghost azeghost 4.4M Jul  1 22:35 pokemon_training.tar.partae


In [28]:
#move to correct folder and push to git
!mkdir /home/azeghost/git/LMDB_Datasets/pokemon
!mv pokemon_training.tar.part* /home/azeghost/git/LMDB_Datasets/pokemon
!ls -la /home/azeghost/git/LMDB_Datasets/pokemon 

mkdir: cannot create directory ‘/home/azeghost/git/LMDB_Datasets/pokemon’: File exists
mv: cannot stat 'pokemon_training.tar.part*': No such file or directory
total 90912
drwxr-xr-x 2 azeghost azeghost     4096 Jul  1 22:36 .
drwxr-xr-x 6 azeghost azeghost     4096 Jul  1 22:31 ..
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partaa
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partab
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partac
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partad
-rw-r--r-- 1 azeghost azeghost  4597619 Jul  1 22:35 pokemon_training.tar.partae
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:30 pokemon_validation.tar.partaa


In [29]:
# push to git
# !cd /home/azeghost/git/LMDB_Datasets
# !git add ../pokemon/*
# # !git status
# !git commit -m "pokemon_training"
# !git push orign master # Check if you have ssh token enabled for bash use otherwise enter login and password

## For validation

In [12]:
!tar -cvjf pokemon_validation.tar.bz2 /home/azeghost/git/Generative_Models/LMDB/_validation/* 

tar: Removing leading `/' from member names
/home/azeghost/git/Generative_Models/LMDB/_validation/data.mdb
/home/azeghost/git/Generative_Models/LMDB/_validation/lock.mdb


In [20]:
!split -b 200M pokemon_training.tar.bz2 "pokemon_validation.tar.part"
!ls -lh pokemon_validation.tar.part*

-rw-r--r-- 1 azeghost azeghost 45M Jul  1 22:30 pokemon_validation.tar.partaa


In [21]:
!mkdir /home/azeghost/git/LMDB_Datasets/pokemon
!mv pokemon_validation.tar.part* /home/azeghost/git/LMDB_Datasets/pokemon
!ls /home/azeghost/git/LMDB_Datasets/pokemon

pokemon_validation.tar.partaa


In [19]:
# !cd /home/azeghost/git/LMDB_Datasets
# !git add pokemon/*
# !git status
# !git commit -m "pokemon_validation"
# !git push orign master

# After downloading from Git combine and unzip

## combine them

In [34]:
#pokemon_training
!cat /home/azeghost/git/LMDB_Datasets/pokemon/pokemon_training.tar.part* >/home/azeghost/git/LMDB_Datasets/pokemon/pokemon_training.tar.bz2
#!cat <split files put * at the end> > <final zip name>
!ls -la /home/azeghost/git/LMDB_Datasets/pokemon

total 181820
drwxr-xr-x 2 azeghost azeghost     4096 Jul  1 22:40 .
drwxr-xr-x 6 azeghost azeghost     4096 Jul  1 22:31 ..
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:40 pokemon_training.tar.bz2
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partaa
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partab
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partac
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partad
-rw-r--r-- 1 azeghost azeghost  4597619 Jul  1 22:35 pokemon_training.tar.partae
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:39 pokemon_validation.tar.bz2
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:30 pokemon_validation.tar.partaa


In [35]:
#pokemon_validation
!cat /home/azeghost/git/LMDB_Datasets/pokemon/pokemon_validation.tar.part* >/home/azeghost/git/LMDB_Datasets/pokemon/pokemon_validation.tar.bz2
!ls -la /home/azeghost/git/LMDB_Datasets/pokemon

total 181816
drwxr-xr-x 2 azeghost azeghost     4096 Jul  1 22:40 .
drwxr-xr-x 6 azeghost azeghost     4096 Jul  1 22:31 ..
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:40 pokemon_training.tar.bz2
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partaa
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partab
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partac
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partad
-rw-r--r-- 1 azeghost azeghost  4597619 Jul  1 22:35 pokemon_training.tar.partae
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:41 pokemon_validation.tar.bz2
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:30 pokemon_validation.tar.partaa


## Unzip   

In [37]:
!tar -xf /home/azeghost/git/LMDB_Datasets/pokemon/pokemon_validation.tar.bz2 -C /home/azeghost/git/LMDB_Datasets
!ls -la /home/azeghost/git/LMDB_Datasets

total 40
drwxr-xr-x 7 azeghost azeghost 4096 Jul  1 22:49 .
drwxr-xr-x 5 azeghost azeghost 4096 Jun 25 08:39 ..
drwxr-xr-x 8 azeghost azeghost 4096 Jul  1 22:31 .git
drwxr-xr-x 3 azeghost azeghost 4096 Jul  1 22:49 home
drwxrwxr-x 3 azeghost azeghost 4096 Jul  1 22:32 .idea
-rw-rw-r-- 1 azeghost azeghost 8376 Jul  1 22:24 LMDB_Write_and_Read_R01.ipynb
drwxr-xr-x 3 azeghost azeghost 4096 Jul  1 22:44 pokemon
drwxr-xr-x 3 azeghost azeghost 4096 Jul  1 22:49 Pokemon


# Read the lmdb 

In [ ]:
def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], []
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            pokemon_image = pickle.loads(data)
            images.append(pokemon_image.get_image())
            labels.append(pokemon_image.label)
    env.close()
    return images, labels

In [ ]:
x, y = read_many_lmdb('./LMDB/_validation', 218)